In [1]:
import pygame
import sys
import os
import random
from moviepy import VideoFileClip  # Importer VideoFileClip depuis moviepy

# Configuration de l'environnement pour MoviePy (si nécessaire)
os.environ["IMAGEIO_FFMPEG_EXE"] = "/opt/homebrew/bin/ffmpeg"

# Constantes globales du jeu
WHITE = (255, 255, 255)  # Couleur blanche en RGB
SCREEN_WIDTH, SCREEN_HEIGHT = 400, 600  # Dimensions de la fenêtre du jeu
FPS = 30  # Images par seconde
GRAVITY = 0.25  # Gravité appliquée au joueur
FLAP_STRENGTH = -5  # Force du saut du joueur
PIPE_SPEED_INIT = -4  # Vitesse initiale des tuyaux
PIPE_GAP_MIN = 180  # Écart minimal entre les tuyaux
PIPE_GAP_MAX = 200  # Écart maximal entre les tuyaux
SCORE_INTERVAL_FOR_SPEED_INCREASE = 3  # Score nécessaire pour augmenter la vitesse

def resource_path(relative_path):
    """
    Obtenir le chemin absolu vers une ressource, fonctionne pour le développement et pour PyInstaller.

    Parameters:
        relative_path (str): Chemin relatif de la ressource.

    Returns:
        str: Chemin absolu vers la ressource.
    """
    try:
        # PyInstaller crée un dossier temporaire et stocke le chemin dans _MEIPASS
        base_path = sys._MEIPASS
    except Exception:
        base_path = os.path.abspath(".")
    return os.path.join(base_path, relative_path)

class Game:
    """
    Classe principale du jeu qui gère la boucle du jeu, les événements, les mises à jour et les rendus.
    """
    def __init__(self):
        pygame.init()  # Initialiser tous les modules Pygame
        pygame.display.set_caption("Joyeux Anniversaire!")  # Définir le titre de la fenêtre
        self.screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))  # Créer la fenêtre du jeu
        self.clock = pygame.time.Clock()  # Créer un objet horloge pour contrôler les FPS
        self.load_resources()  # Charger toutes les ressources nécessaires
        self.running = True  # Indique si le jeu est en cours d'exécution
        self.video_played = False  # Indique si la vidéo a été jouée
        self.score = 0  # Score du joueur
        self.last_speed_increase_score = 0  # Dernier score où la vitesse a augmenté
        self.font_name = self.font_path  # Nom de la police utilisée

        # Créer les objets du jeu
        self.player = Player(self)  # Créer le joueur
        self.pipes = []  # Liste des tuyaux à l'écran
        self.clouds = [Cloud(self) for _ in range(5)]  # Liste des nuages
        self.stars = [Star() for _ in range(25)]  # Liste des étoiles

    def load_resources(self):
        """
        Charger toutes les ressources nécessaires (images, polices).
        """
        # Chemin vers la police utilisée
        self.font_path = resource_path('resources/SuperMario256.ttf')

        # Charger et redimensionner les images nécessaires
        self.bird_img = pygame.image.load(resource_path('resources/mario_volant.png')).convert_alpha()
        self.bird_img = pygame.transform.scale(self.bird_img, (50, 50))

        self.brique_img = pygame.image.load(resource_path('resources/brique.png')).convert_alpha()
        self.brique_img = pygame.transform.scale(self.brique_img, (50, 50))

        self.plante_img = pygame.image.load(resource_path('resources/plante.png')).convert_alpha()
        self.plante_img = pygame.transform.scale(self.plante_img, (50, 75))

        self.nuage_img = pygame.image.load(resource_path('resources/nuage.png')).convert_alpha()
        self.nuage_img = pygame.transform.scale(self.nuage_img, (100, 60))

        self.title_image = pygame.image.load(resource_path('resources/mission_joyeux_anniversaire.png')).convert_alpha()
        self.game_over_image = pygame.image.load(resource_path('resources/Game_over.png')).convert_alpha()

        # Redimensionner les images de titre et de fin
        image_width = 300
        title_image_height = int(self.title_image.get_height() * (image_width / self.title_image.get_width()))
        self.title_image = pygame.transform.scale(self.title_image, (image_width, title_image_height))
        game_over_image_height = int(self.game_over_image.get_height() * (image_width / self.game_over_image.get_width()))
        self.game_over_image = pygame.transform.scale(self.game_over_image, (image_width, game_over_image_height))

    def draw_text(self, surf, text, size, x, y, color=(255, 255, 255)):
        """
        Dessiner du texte sur une surface donnée.

        Parameters:
            surf (Surface): Surface sur laquelle dessiner le texte.
            text (str): Texte à afficher.
            size (int): Taille de la police.
            x (int): Position x du texte.
            y (int): Position y du texte.
            color (tuple): Couleur du texte en RGB.
        """
        font = pygame.font.Font(self.font_name, size)  # Créer un objet police
        text_surface = font.render(text, True, color)  # Rendre le texte
        text_rect = text_surface.get_rect()  # Obtenir le rectangle du texte
        text_rect.midtop = (x, y)  # Positionner le rectangle
        surf.blit(text_surface, text_rect)  # Dessiner le texte sur la surface

    def run(self):
        """
        Lancer le jeu, gérer la boucle principale et les écrans de démarrage et de fin.
        """
        self.start_screen()  # Afficher l'écran de démarrage
        while self.running:
            self.new_game()  # Initialiser une nouvelle partie
            self.game_loop()  # Lancer la boucle de jeu
            if not self.game_over_screen():  # Afficher l'écran de fin de jeu
                self.running = False  # Quitter le jeu si le joueur ne veut pas rejouer
        pygame.quit()  # Quitter Pygame
        sys.exit()  # Quitter le programme

    def new_game(self):
        """
        Initialiser une nouvelle partie en réinitialisant les variables et en créant les objets du jeu.
        """
        self.player.reset()  # Réinitialiser le joueur
        self.score = 0  # Réinitialiser le score
        self.video_played = False  # Réinitialiser l'indicateur de vidéo jouée
        self.pipes = []  # Réinitialiser les tuyaux
        self.clouds = [Cloud(self) for _ in range(5)]  # Recréer les nuages
        self.stars = [Star() for _ in range(25)]  # Recréer les étoiles
        self.create_initial_pipes()  # Créer le premier tuyau

    def create_initial_pipes(self):
        """
        Créer le premier tuyau avec un délai pour que le joueur ait le temps de se préparer.
        """
        delay_seconds = 3  # Délai en secondes avant l'apparition du premier tuyau
        pipe_speed_per_second = -PIPE_SPEED_INIT * FPS  # Calcul de la vitesse du tuyau par seconde
        distance = pipe_speed_per_second * delay_seconds  # Distance que le tuyau doit parcourir
        initial_pipe_x = SCREEN_WIDTH + distance  # Position x initiale du premier tuyau
        self.pipes.append(Pipe(self, initial_pipe_x))  # Ajouter le tuyau à la liste

    def game_loop(self):
        """
        Boucle principale du jeu où les événements, les mises à jour et les rendus sont gérés.
        """
        self.last_speed_increase_score = 0  # Réinitialiser le score de la dernière augmentation de vitesse
        running = True  # Indicateur pour continuer la boucle
        while running:
            self.clock.tick(FPS)  # Contrôler les FPS
            self.handle_events()  # Gérer les événements
            self.update()  # Mettre à jour les objets du jeu
            self.draw()  # Dessiner les objets à l'écran
            if self.player.dead:  # Vérifier si le joueur est mort
                running = False  # Sortir de la boucle de jeu

    def handle_events(self):
        """
        Gérer les événements tels que les entrées du clavier ou la fermeture de la fenêtre.
        """
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.running = False  # Arrêter le jeu
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:
                    self.player.flap()  # Le joueur saute
                elif event.key == pygame.K_DOWN:
                    self.player.dive()  # Le joueur plonge

    def update(self):
        """
        Mettre à jour l'état du jeu, y compris le joueur, les tuyaux, les nuages et la détection des collisions.
        """
        self.player.update()  # Mettre à jour le joueur
        for cloud in self.clouds:
            cloud.update()  # Mettre à jour les nuages
        for pipe in self.pipes:
            pipe.update()  # Mettre à jour les tuyaux
            if pipe.off_screen():
                self.pipes.remove(pipe)  # Supprimer le tuyau s'il est hors de l'écran
                self.pipes.append(Pipe(self))  # Ajouter un nouveau tuyau
            if not pipe.passed and pipe.x + pipe.width < self.player.x:
                self.score += 1  # Incrémenter le score si le joueur passe le tuyau
                pipe.passed = True  # Marquer le tuyau comme passé

        # Augmenter la vitesse des tuyaux en fonction du score
        if self.score // SCORE_INTERVAL_FOR_SPEED_INCREASE > self.last_speed_increase_score:
            for pipe in self.pipes:
                pipe.speed -= 1  # Augmenter la vitesse (négative) des tuyaux
            self.last_speed_increase_score = self.score // SCORE_INTERVAL_FOR_SPEED_INCREASE

        # Vérifier les collisions entre le joueur et les tuyaux
        for pipe in self.pipes:
            if self.player.collide_with(pipe):
                self.player.dead = True  # Le joueur est mort s'il y a collision

        # Vérifier si le joueur est sorti de l'écran
        if self.player.y >= SCREEN_HEIGHT - self.player.height or self.player.y <= 0:
            self.player.dead = True  # Le joueur est mort s'il sort de l'écran

        # Jouer la vidéo si le score atteint 10 et que la vidéo n'a pas encore été jouée
        if self.score >= 10 and not self.video_played:
            self.play_video(resource_path('resources/test_video.avi'))
            self.video_played = True

    def draw(self):
        """
        Dessiner tous les éléments du jeu à l'écran.
        """
        self.screen.fill((102, 190, 209))  # Remplir l'écran avec une couleur de fond
        for star in self.stars:
            star.draw(self.screen)  # Dessiner les étoiles
        for cloud in self.clouds:
            cloud.draw(self.screen)  # Dessiner les nuages
        self.player.draw(self.screen)  # Dessiner le joueur
        for pipe in self.pipes:
            pipe.draw(self.screen)  # Dessiner les tuyaux
        self.draw_text(self.screen, f"Score: {self.score}", 24, SCREEN_WIDTH - 100, 50)  # Afficher le score
        pygame.display.update()  # Mettre à jour l'affichage

    def start_screen(self):
        """
        Afficher l'écran de démarrage du jeu avec les instructions et un bouton pour commencer.
        """
        intro_text = [
            ("Bienvenue au jeu d'anniversaire!", 20, (255, 255, 255)),
            ("Aidez Mario à atteindre", 20, (255, 255, 255)),
            ("le score de 10 pour une surprise spéciale!", 20, (255, 255, 255)),
            ("", 20),  # Ligne vide pour l'espacement
            ("Utilisez la flèche du haut pour monter,", 20, (255, 255, 255)),
            ("et celle du bas pour descendre.", 20, (255, 255, 255)),
            ("Bonne chance!", 20, (255, 255, 255))
        ]
        buttons = [
            Button("Commencer", 125, 450, 150, 50, (126, 223, 71))
        ]
        self.show_screen(self.title_image, intro_text, buttons)

    def game_over_screen(self):
        """
        Afficher l'écran de fin de jeu avec le score final et des options pour rejouer ou quitter.

        Returns:
            bool: True si le joueur veut rejouer, False sinon.
        """
        score_text = f"Score Final: {self.score}"
        text_lines_with_styles = [
            (score_text, 36, (237, 237, 237)),  # Rouge pour le texte du score
        ]
        buttons = [
            Button("Rejouer", 100, 450, 100, 50, (81, 219, 63), (237, 237, 237)),  # Texte noir sur fond vert clair
            Button("Quitter", 200, 450, 100, 50, (247, 46, 46), (237, 237, 237))   # Texte noir sur fond rouge clair
        ]
        return self.show_screen(self.game_over_image, text_lines_with_styles, buttons)

    def show_screen(self, image, text_lines_with_styles, buttons, default_size=15, default_color=(255, 255, 255)):
        """
        Afficher un écran générique avec une image, du texte (avec tailles et couleurs) et des boutons.

        Parameters:
            image (Surface): Image à afficher en haut de l'écran.
            text_lines_with_styles (list): Liste de tuples contenant le texte, la taille de police et la couleur.
            buttons (list): Liste des objets Button à afficher.
            default_size (int): Taille de police par défaut si aucune taille n'est spécifiée.
            default_color (tuple): Couleur par défaut si aucune couleur n'est spécifiée.

        Returns:
            bool: True si le joueur sélectionne "Rejouer", False s'il sélectionne "Quitter".
        """
        waiting = True  # Indicateur pour rester sur l'écran
        while waiting:
            self.clock.tick(15)  # Contrôler les FPS de l'écran
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    for button in buttons:
                        if button.is_clicked():
                            if button.text == "Commencer":
                                return
                            elif button.text == "Rejouer":
                                return True
                            elif button.text == "Quitter":
                                pygame.quit()
                                sys.exit()
            self.screen.fill((102, 190, 209))  # Remplir l'écran avec une couleur de fond
            for star in self.stars:
                star.draw(self.screen)  # Dessiner les étoiles
            for cloud in self.clouds:
                cloud.update()  # Mettre à jour les nuages
                cloud.draw(self.screen)  # Dessiner les nuages
            self.screen.blit(image, (SCREEN_WIDTH / 2 - image.get_width() / 2, SCREEN_HEIGHT / 10))  # Afficher l'image
            y_offset = SCREEN_HEIGHT / 3  # Position y initiale du texte
            for item in text_lines_with_styles:
                if isinstance(item, tuple):
                    # Récupérer le texte, la taille et la couleur
                    line = item[0]
                    size = item[1] if len(item) > 1 else default_size
                    color = item[2] if len(item) > 2 else default_color
                else:
                    # Si aucun style n'est fourni, utiliser les valeurs par défaut
                    line = item
                    size = default_size
                    color = default_color
                self.draw_text(self.screen, line, size, SCREEN_WIDTH / 2, y_offset, color)  # Dessiner le texte avec la couleur spécifiée
                y_offset += size + 5  # Ajouter un espace entre les lignes
            for button in buttons:
                button.draw(self.screen, font_name=self.font_name)  # Dessiner chaque bouton avec la police du jeu
            pygame.display.flip()  # Mettre à jour l'affichage

    def play_video(self, video_path):
        """
        Jouer une vidéo en plein écran.

        Parameters:
            video_path (str): Chemin vers la vidéo à jouer.
        """
        clip = VideoFileClip(video_path)  # Charger la vidéo
        clip = clip.resized(height=SCREEN_HEIGHT, width=SCREEN_WIDTH)  # Redimensionner la vidéo
        clock = pygame.time.Clock()  # Créer un objet horloge pour contrôler les FPS
        for frame in clip.iter_frames(fps=30, dtype="uint8"):
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
            frame_surface = pygame.surfarray.make_surface(frame.swapaxes(0, 1))  # Créer une surface à partir du frame
            self.screen.blit(frame_surface, (0, 0))  # Dessiner le frame sur l'écran
            pygame.display.update()  # Mettre à jour l'affichage
            clock.tick(30)  # Contrôler les FPS de la vidéo
        clip.close()  # Fermer le clip vidéo

class Player:
    """
    Classe représentant le joueur (Mario volant).
    """
    def __init__(self, game):
        self.game = game  # Référence à l'objet Game
        self.image = game.bird_img  # Image du joueur
        self.width = self.image.get_width()  # Largeur du joueur
        self.height = self.image.get_height()  # Hauteur du joueur
        self.x = 50  # Position x du joueur
        self.reset()  # Initialiser les autres attributs

    def reset(self):
        """
        Réinitialiser la position et l'état du joueur.
        """
        self.y = SCREEN_HEIGHT // 2  # Position y initiale du joueur
        self.velocity = 0  # Vitesse verticale du joueur
        self.dead = False  # Indicateur si le joueur est mort

    def flap(self):
        """
        Faire sauter le joueur en lui appliquant une force vers le haut.
        """
        self.velocity = FLAP_STRENGTH  # Appliquer la force du saut

    def dive(self):
        """
        Faire plonger le joueur en lui appliquant une force vers le bas.
        """
        self.velocity = -FLAP_STRENGTH  # Appliquer la force de la plongée

    def update(self):
        """
        Mettre à jour la position du joueur en fonction de la gravité et de sa vitesse.
        """
        self.velocity += GRAVITY  # Appliquer la gravité à la vitesse
        self.y += self.velocity  # Mettre à jour la position y du joueur

    def draw(self, screen):
        """
        Dessiner le joueur à l'écran.

        Parameters:
            screen (Surface): Surface sur laquelle dessiner le joueur.
        """
        screen.blit(self.image, (self.x, self.y))  # Dessiner l'image du joueur

    def get_rect(self):
        """
        Obtenir le rectangle englobant du joueur pour la détection des collisions.

        Returns:
            Rect: Rectangle englobant du joueur.
        """
        return pygame.Rect(self.x, self.y, self.width, self.height)

    def collide_with(self, pipe):
        """
        Vérifier la collision avec un tuyau.

        Parameters:
            pipe (Pipe): Tuyau avec lequel vérifier la collision.

        Returns:
            bool: True si collision, False sinon.
        """
        player_rect = self.get_rect()  # Obtenir le rectangle du joueur
        collision = player_rect.colliderect(pipe.upper_pipe_rect) or \
                    player_rect.colliderect(pipe.lower_pipe_rect)  # Vérifier collision avec les tuyaux
        if pipe.has_plant and pipe.plant_rect:
            collision = collision or player_rect.colliderect(pipe.plant_rect)  # Vérifier collision avec la plante
        return collision  # Retourner le résultat de la collision

class Pipe:
    """
    Classe représentant un tuyau dans le jeu.
    """
    def __init__(self, game, x=None):
        self.game = game  # Référence à l'objet Game
        self.image = game.brique_img  # Image de la brique du tuyau
        self.plant_image = game.plante_img  # Image de la plante
        self.width = self.image.get_width()  # Largeur du tuyau
        self.height = self.image.get_height()  # Hauteur du tuyau
        self.x = x if x is not None else SCREEN_WIDTH  # Position x du tuyau
        self.speed = PIPE_SPEED_INIT  # Vitesse du tuyau
        self.pipe_gap = random.randint(PIPE_GAP_MIN, PIPE_GAP_MAX)  # Écart entre les tuyaux
        self.pipe_y = random.randint(100 + self.pipe_gap, SCREEN_HEIGHT - 50)  # Position y du tuyau inférieur
        self.has_plant = random.choice([True, False])  # Indicateur si le tuyau supérieur a une plante
        self.passed = False  # Indicateur si le joueur a passé le tuyau

        # Initialisation des rectangles de collision
        self.upper_pipe_rect = None  # Rectangle de collision du tuyau supérieur
        self.lower_pipe_rect = None  # Rectangle de collision du tuyau inférieur
        self.plant_rect = None  # Rectangle de collision de la plante
        self.calculate_collision_rects()  # Calculer les rectangles de collision

    def calculate_collision_rects(self):
        """
        Calculer les rectangles de collision pour le tuyau supérieur, le tuyau inférieur et la plante.
        """
        # Calcul du rectangle du tuyau supérieur
        y_top = self.pipe_y - self.pipe_gap - self.height  # Position y du haut du tuyau supérieur
        min_y_top = -self.height  # Position minimale y du tuyau supérieur
        if self.has_plant:
            min_y_top += 2 * self.height  # Ajuster si une plante est présente
        upper_pipe_height = y_top - min_y_top  # Hauteur du tuyau supérieur
        self.upper_pipe_rect = pygame.Rect(
            self.x, min_y_top, self.width, upper_pipe_height
        )

        # Calcul du rectangle du tuyau inférieur
        y_bottom = self.pipe_y  # Position y du tuyau inférieur
        lower_pipe_height = SCREEN_HEIGHT - y_bottom  # Hauteur du tuyau inférieur
        self.lower_pipe_rect = pygame.Rect(
            self.x, y_bottom, self.width, lower_pipe_height
        )

        # Calcul du rectangle de la plante
        if self.has_plant:
            plant_x = self.x + (self.width - self.plant_image.get_width()) // 2  # Position x de la plante
            plant_y = self.pipe_y - self.plant_image.get_height()  # Position y de la plante
            plant_width = self.plant_image.get_width()  # Largeur de la plante
            plant_height = self.plant_image.get_height()  # Hauteur de la plante
            self.plant_rect = pygame.Rect(
                plant_x, plant_y, plant_width, plant_height
            )
        else:
            self.plant_rect = None  # Pas de plante, donc pas de rectangle de collision

    def update(self):
        """
        Mettre à jour la position du tuyau et recalculer les rectangles de collision.
        """
        self.x += self.speed  # Déplacer le tuyau vers la gauche
        self.calculate_collision_rects()  # Recalculer les rectangles de collision

    def off_screen(self):
        """
        Vérifier si le tuyau est sorti de l'écran.

        Returns:
            bool: True si le tuyau est hors de l'écran, False sinon.
        """
        return self.x < -self.width  # Le tuyau est hors écran s'il est entièrement à gauche

    def draw(self, screen):
        """
        Dessiner le tuyau et la plante (si présente) à l'écran.

        Parameters:
            screen (Surface): Surface sur laquelle dessiner le tuyau.
        """
        # Dessiner le tuyau supérieur
        y = self.upper_pipe_rect.bottom - self.height  # Position y initiale pour dessiner les briques du tuyau supérieur
        while y >= self.upper_pipe_rect.top:
            screen.blit(pygame.transform.flip(self.image, False, True), (self.x, y))  # Dessiner la brique
            y -= self.height  # Monter pour dessiner la prochaine brique

        # Dessiner le tuyau inférieur
        y = self.lower_pipe_rect.top  # Position y initiale pour dessiner les briques du tuyau inférieur
        while y < SCREEN_HEIGHT:
            screen.blit(self.image, (self.x, y))  # Dessiner la brique
            y += self.height  # Descendre pour dessiner la prochaine brique

        # Dessiner la plante si elle est présente
        if self.has_plant and self.plant_rect:
            screen.blit(self.plant_image, (self.plant_rect.x, self.plant_rect.y))  # Dessiner la plante

class Cloud:
    """
    Classe représentant un nuage en arrière-plan.
    """
    def __init__(self, game):
        self.game = game  # Référence à l'objet Game
        self.image = game.nuage_img  # Image du nuage
        self.width = self.image.get_width()  # Largeur du nuage
        self.height = self.image.get_height()  # Hauteur du nuage
        self.x = random.randint(0, SCREEN_WIDTH)  # Position x initiale aléatoire
        self.y = random.randint(0, SCREEN_HEIGHT // 2)  # Position y initiale aléatoire
        self.speed = random.uniform(0.1, 2)  # Vitesse de déplacement du nuage

    def update(self):
        """
        Mettre à jour la position du nuage.
        """
        self.x -= self.speed  # Déplacer le nuage vers la gauche
        if self.x < -self.width:
            self.x = SCREEN_WIDTH  # Réinitialiser la position x du nuage
            self.y = random.randint(0, SCREEN_HEIGHT // 2)  # Réinitialiser la position y
            self.speed = random.uniform(0.1, 2)  # Réinitialiser la vitesse

    def draw(self, screen):
        """
        Dessiner le nuage à l'écran.

        Parameters:
            screen (Surface): Surface sur laquelle dessiner le nuage.
        """
        screen.blit(self.image, (self.x, self.y))  # Dessiner l'image du nuage

class Star:
    """
    Classe représentant une étoile en arrière-plan.
    """
    def __init__(self):
        self.x = random.randint(0, SCREEN_WIDTH)  # Position x aléatoire
        self.y = random.randint(0, SCREEN_HEIGHT)  # Position y aléatoire
        self.radius = random.randint(1, 3)  # Rayon de l'étoile
        self.color = (255, 255, 0)  # Couleur de l'étoile (jaune)

    def draw(self, screen):
        """
        Dessiner l'étoile à l'écran.

        Parameters:
            screen (Surface): Surface sur laquelle dessiner l'étoile.
        """
        pygame.draw.circle(screen, self.color, (self.x, self.y), self.radius)  # Dessiner l'étoile

class Button:
    """
    Représente un bouton interactif dans l'interface utilisateur.

    Attributes:
        text (str): Le texte affiché sur le bouton.
        x (int): La position x du bouton.
        y (int): La position y du bouton.
        width (int): La largeur du bouton.
        height (int): La hauteur du bouton.
        color (tuple): La couleur de fond du bouton.
        text_color (tuple): La couleur du texte affiché sur le bouton.
    """

    def __init__(self, text, x, y, width, height, color=(170, 170, 170), text_color=(255, 255, 255)):
        self.text = text  # Texte du bouton
        self.x = x  # Position x du bouton
        self.y = y  # Position y du bouton
        self.width = width  # Largeur du bouton
        self.height = height  # Hauteur du bouton
        self.color = color  # Couleur du bouton
        self.text_color = text_color  # Couleur du texte

    def draw(self, screen, font_name, font_size=20):
        """
        Dessine le bouton sur l'écran.

        Parameters:
            screen (Surface): La surface Pygame sur laquelle dessiner le bouton.
            font_name (str): Le chemin de la police utilisée pour le texte du bouton.
            font_size (int): La taille de la police.
        """
        mouse = pygame.mouse.get_pos()  # Obtenir la position de la souris
        click = pygame.mouse.get_pressed()  # Obtenir l'état des boutons de la souris

        # Détecter si la souris survole le bouton
        if self.is_hovered(mouse):
            pygame.draw.rect(screen, self.color, (self.x, self.y, self.width, self.height))
        else:
            pygame.draw.rect(screen, (200, 200, 200), (self.x, self.y, self.width, self.height))

        # Dessiner le texte du bouton
        font = pygame.font.Font(font_name, font_size)
        text_surface = font.render(self.text, True, self.text_color)
        text_rect = text_surface.get_rect(center=(self.x + self.width / 2, self.y + self.height / 2))
        screen.blit(text_surface, text_rect)

    def is_hovered(self, mouse):
        """
        Vérifie si la souris survole le bouton.

        Parameters:
            mouse (tuple): Position actuelle de la souris.

        Returns:
            bool: True si la souris survole le bouton, sinon False.
        """
        return self.x < mouse[0] < self.x + self.width and self.y < mouse[1] < self.y + self.height

    def is_clicked(self):
        """
        Vérifie si le bouton est cliqué.

        Returns:
            bool: True si le bouton est cliqué, sinon False.
        """
        return self.is_hovered(pygame.mouse.get_pos()) and pygame.mouse.get_pressed()[0]

def main():
    """
    Fonction principale pour lancer le jeu.
    """
    game = Game()  # Créer une instance du jeu
    game.run()  # Lancer le jeu

if __name__ == "__main__":
    main()  # Appeler la fonction principale si le script est exécuté directement

pygame 2.6.1 (SDL 2.28.4, Python 3.11.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

/Users/souchaud/anaconda3/envs/flappy_bird/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
